## B09705039_HW02

In [1]:
# import module
from nltk.stem import PorterStemmer
import math
import numpy as np

In [2]:
# All tokenized doc result
doc_amount = 1095
all_doc = []
for doc in range(1, doc_amount + 1):
    # Read file.
    path = "./data/" + str(doc) + ".txt"
    f = open(path, 'r')
    all_text = f.read()
    f.close()

    # Tokenization.
    # signs that can be ignored: We only listed a few here, if needed more we can add more.
    nonAlphanumeric = [",", "'", ";", ":", '"', "@", "!", "?", "(", ")", "[", "]", "<", ">", "=", "+", "^", "$", "~", "*", "/", "{", "}", "&", "#", "%","`", "_"]
    for i in nonAlphanumeric:
        all_text = all_text.replace(i, " ")

    # periods: concatenate
    all_text = all_text.replace(".", "")
    # hyphens: concatenate
    all_text = all_text.replace("-", "")
    
    # remove digits
    all_text = ''.join([i for i in all_text if not i.isdigit()])

    tokenize = all_text.split()

    # Lowercasing everything.
    lowercase = []
    for i in tokenize:
        lowercase.append(i.lower())
        
    # Stopword removal.
    # Read stopwords file. stopwords.txt is generated by nltk.
    path = 'stopwords.txt'
    f2 = open(path, 'r')
    stop_words = f2.read()
    f2.close()
    
    # Removal start.
    stop_words = stop_words.split()
    stopword_removed = []
    for w in lowercase:
        if w not in stop_words:
            stopword_removed.append(w)
        
    # Stemming using Porter’s algorithm.
    ps = PorterStemmer()

    after_stemming = []
    for w in stopword_removed:
        after_stemming.append(ps.stem(w))
        
    all_doc.append(after_stemming)

In [3]:
# get tf, df dict
tf_dict_arr = []
df_dict = {}
for i in range(doc_amount):
    # count tf
    temp_tf_dict = {}
    for j in all_doc[i]:
        if j not in temp_tf_dict.keys():
            temp_tf_dict[j] = 1
        else:
            temp_tf_dict[j] += 1
    tf_dict_arr.append(temp_tf_dict)
    # count df
    for x in temp_tf_dict.keys():
        if x not in df_dict.keys():
            df_dict[x] = 1
        else:
            df_dict[x] += 1

In [4]:
# Output dictionary.txt
path = 'dictionary.txt'
f = open(path, 'w')

sortednames = sorted(df_dict.keys(), key=lambda x:x.lower())
writeArr = []
count = 1
writeArr.append("t_index " + "term " + "df" + "\n")
for key in sortednames:
    writeArr.append(str(count)+ " ")
    writeArr.append(key + " ")
    writeArr.append(str(df_dict[key]) + "\n")
    count += 1
f.writelines(writeArr)
f.close()

In [5]:
# Count tf-idf unit vector
# Calculate idf
idf = {}
for i in sortednames:
    idf[i] = math.log10(doc_amount/df_dict[i])
# Calculate each document's tf-idf unit vector
tf_idf_norm = []
for i in tf_dict_arr:
    tf_idf_temp = {}
    norm = 0
    for j in i.keys():
        tf_idf_temp[sortednames.index(j)] = i[j] * idf[j]
        norm += (tf_idf_temp[sortednames.index(j)]) ** 2
    norm = math.sqrt(norm)
    for j in i.keys():
        tf_idf_temp[sortednames.index(j)] = tf_idf_temp[sortednames.index(j)] / norm
    tf_idf_norm.append(tf_idf_temp)

In [6]:
# Output docID.txt
for doc in range(1, doc_amount + 1):
    path = "./output/doc" + str(doc) + ".txt"
    f = open(path, 'w')

    sortednames = sorted(tf_idf_norm[doc - 1].keys(), key=lambda x:x)
    writeArr = []
    writeArr.append(str(len(sortednames)) + "\n")
    writeArr.append("t_index " + "tf-idf" + "\n")
    for key in sortednames:
        writeArr.append(str(key + 1) + " ")
        writeArr.append(str(tf_idf_norm[doc - 1][key]) + "\n")
    f.writelines(writeArr)
    f.close()

In [7]:
# Count cosine similarity
def cosine(Docx, Docy):
    cs = 0
    for i in Docx.keys():
        if i in Docy.keys():
            cs += (Docx[i] * Docy[i])
    return cs

Docx = tf_idf_norm[0]
Docy = tf_idf_norm[1]
cs = cosine(Docx, Docy)
print("The cosine similarity between document 1 and 2 is", cs)

The cosine similarity between document 1 and 2 is 0.18056710715588556
